In [1]:
import os
import sklearn
import pathlib
import numpy as np
import scipy as sp
import pandas as pd

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
dataset_path = pathlib.Path('/tf/sentiment_prediction/utils/kaggle_previous_10.csv')

In [4]:
dataset = pd.read_csv(dataset_path)

In [5]:
y = dataset['stock_move'].tolist()

In [6]:
x = [[float(x) for x in line.split(',')] for line in dataset['prevois_stock_moves'].tolist()]

In [7]:
y = np.array(y)
y.shape

(3408,)

In [8]:
x = np.array(x)
x.shape

(3408, 250)

In [9]:
np.mean(y), np.var(y)

(0.3160211267605634, 0.2161517742015473)

In [10]:
indices_0 = np.argwhere(y == 0)
indices_1 = np.argwhere(y == 1)

In [11]:
print('Zeros:', indices_0.shape[0])
print('Ones:', indices_1.shape[0])

Zeros: 2331
Ones: 1077


In [12]:
indices_0 = indices_0[:indices_1.shape[0]]

In [13]:
print(indices_0.shape)
print(indices_1.shape)

(1077, 1)
(1077, 1)


In [14]:
x = np.concatenate((x[indices_0], x[indices_1]), axis=0).squeeze()
x.shape

(2154, 250)

In [15]:
y = np.concatenate([y[indices_0], y[indices_1]], axis=0).squeeze()
y.shape

(2154,)

In [16]:
np.sum(x), np.mean(x), np.var(x)

(228554.49110772857, 0.4244280243411858, 0.19886200788875572)

In [17]:
np.sum(y), np.mean(y), np.var(y)

(1077, 0.5, 0.25)

# Keras model

In [18]:
import keras

Using TensorFlow backend.


In [19]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(16, activation='relu', input_shape=(x.shape[1],)))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
model.compile(
    optimizer=keras.optimizers.Adam(0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                4016      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 4,433
Trainable params: 4,433
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(
    x,
    y,
    epochs=100,
    shuffle=True,
    batch_size=100,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping('val_loss', mode='min', patience=20, verbose=1)
    ]
)

Instructions for updating:
Use tf.cast instead.
Train on 1723 samples, validate on 431 samples
Epoch 1/100
1723/1723 [==============================] - 3s 2ms/step - loss: 0.7211 - acc: 0.3755 - val_loss: 0.6248 - val_acc: 0.9629
Epoch 2/100
1723/1723 [==============================] - 0s 64us/step - loss: 0.7109 - acc: 0.3767 - val_loss: 0.6477 - val_acc: 0.8979
Epoch 3/100
1723/1723 [==============================] - 0s 61us/step - loss: 0.7043 - acc: 0.4063 - val_loss: 0.6623 - val_acc: 0.8097
Epoch 4/100
1723/1723 [==============================] - 0s 59us/step - loss: 0.7004 - acc: 0.4353 - val_loss: 0.6733 - val_acc: 0.6589
Epoch 5/100
1723/1723 [==============================] - 0s 56us/step - loss: 0.6976 - acc: 0.4701 - val_loss: 0.6802 - val_acc: 0.5522
Epoch 6/100
1723/1723 [==============================] - 0s 60us/step - loss: 0.6957 - acc: 0.4962 - val_loss: 0.6857 - val_acc: 0.4664
Epoch 7/100
1723/1723 [==============================] - 0s 61us/step - loss: 0.6944 - acc